In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
!pip install datasets boto3

In [2]:
%%writefile ~/.passwd-s3fs
# secrets

Writing /root/.passwd-s3fs


In [3]:
!chmod 600 ~/.passwd-s3fs
!apt install s3fs
!mkdir /s3
!s3fs baj40ja0abjabucketihvp /s3
!ls /s3/ihvp

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  mailcap mime-support
The following NEW packages will be installed:
  mailcap mime-support s3fs
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 325 kB of archives.
After this operation, 901 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 s3fs amd64 1.90-1 [297 kB]
Fetched 325 kB in 0s (896 kB/s)
Selecting previously unselected package mailcap.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../mailcap_3.70+nmu1ubuntu1_all.deb ...
Unpacking mailcap (3.70+nmu1ubuntu1) ...
Selecting previously unselected package mime-support.
Preparing

In [4]:
!cp /s3/ihvp/TinyStories_ihvp_33000000.pt /content/

In [5]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import torch

# Initialize the tokenizer
# tokenizer.pad_token = tokenizer.eos_token

# https://huggingface.co/datasets/Skylion007/openwebtext
from datasets import load_dataset

class TinyStoriesDataset(Dataset): # IterableDataset
    def __init__(self, split, tokenizer, path='roneneldan/TinyStories', block_size=1024, seed=42, take=10000, skip=0):
        self.split = split
        self.block_size = block_size
        self.tokenizer = tokenizer
        self.dataset = load_dataset(path, split=split, streaming=False).shuffle(seed=seed) #.skip(skip).take(take).with_format('torch')

    def __iter__(self):
        for item in self.dataset:
            text = item['text']
            encoding = self.tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True, padding='max_length', truncation=True, max_length=self.block_size)
            input_ids = encoding['input_ids'].squeeze()

            # Shift the input_ids and attention_mask to the right and pad
            labels = input_ids.clone()
            labels[:-1] = input_ids[1:]
            labels[-1] = self.tokenizer.pad_token_id

            yield {'input_ids': input_ids}, {'input_ids': labels}

    def __getitem__(self, index):
        item = self.dataset[index]
        text = item['text']
        encoding = self.tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True, padding='max_length', truncation=True, max_length=self.block_size)
        input_ids = encoding['input_ids'].squeeze()

        # Shift the input_ids and attention_mask to the right and pad
        labels = input_ids.clone()
        labels[:-1] = input_ids[1:]
        labels[-1] = self.tokenizer.pad_token_id

        return {'input_ids': input_ids}, {'input_ids': labels}

    def get_vocab_size(self):
        return self.tokenizer.vocab_size

    def get_block_size(self):
        return self.block_size


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token
vocab_size = tokenizer.vocab_size

model = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-1M")

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

model.to(device)

# 1. get a_l-1, use forward hook to save input to a layer l during the forward pass
layer_inputs = {}

# TODO something going on with the shapes
def forward_hook_fn(module, input):
    if isinstance(module, nn.Linear):
        layer_inputs[module] = torch.cat([
                input[0],
                torch.ones((input[0].shape[0], input[0].shape[1], 1)).to(input[0].device),
        ], dim=-1).clone().detach()

# 2. get grad_loss, gradients of loss wrt output of linear transformation W_l a_l-1
#    using a backward hook on the linear layer that saves the gradient wrt the linear layer's output

layer_grads = {}

def back_hook_fn(module, grad_input, grad_output):
    if isinstance(module, nn.Linear):
        layer_grads[module] = grad_output[0].clone().detach()

linear_layers = []
for name, module in model.named_modules():
    if isinstance(module, nn.Linear) and 'mlp' in name: # out_proj ???
        # grab linear layers everytime
        linear_layers.append(module)

linear_layers = linear_layers[:-1]  # remove output token logits layer from calculations

tokenizer_config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/48.6M [00:00<?, ?B/s]

In [7]:
def compute_grads(model: nn.Module, train_dataset: DataLoader):

    grads = [[] for _ in range(len(linear_layers))]
    for X, Y in train_dataset:
        model.zero_grad()

        x_ids = X['input_ids'].to(device)
        y_ids = Y['input_ids'].to(device)

        if len(x_ids.shape) == 3:
            x_ids = x_ids.squeeze(1)
            y_ids = y_ids.squeeze(1)

        if len(x_ids.shape) == 1:
            x_ids = x_ids.unsqueeze(0)
            y_ids = y_ids.unsqueeze(0)

        output = model(x_ids, labels=y_ids)
        logits, loss = output['logits'], output['loss']

        loss.backward()
        for i, module in enumerate(linear_layers):
            w_grad = module.weight.grad
            if module.bias is not None:
                b_grad = module.bias.grad.unsqueeze(-1)
                full_grad = torch.cat([w_grad, b_grad], dim=-1)
            else:
                full_grad = torch.cat(
                    [w_grad, torch.zeros([w_grad.shape[0], 1])],
                    dim=-1
                )
            grads[i].append(full_grad)

    return grads

In [8]:
prompt = "Once upon a time, there was a girl name Alice. Alice enjoyed many things. One of the things Alice enjoyed was playing outside."
prompt_enc = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(prompt_enc, max_length=300)
print(tokenizer.decode(out[0]))
z_m_ids = torch.nn.functional.pad(out[0], (0, 2048 - len(out[0])), value=50256)

z_m_label = z_m_ids.clone()
z_m_label[:-1] = z_m_ids[1:]
z_m_label[-1] = tokenizer.pad_token_id

z_m = [({'input_ids': z_m_ids}, {'input_ids': z_m_label})]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, there was a girl name Alice. Alice enjoyed many things. One of the things Alice enjoyed was playing outside. She was so excited to see what was inside.

Alice asked her mom, "Mommy, what is that?" Her mom replied, "It's a secret. It's a secret place to find it."

Alice was so excited. She opened the door and saw a big, beautiful flower. It was a beautiful flower. She was so excited to see it.

Alice and her mom went to the flower. It was so pretty and beautiful. Alice was so happy to see it. She couldn't wait to see it.

Alice and her mom went to the flower and saw that it was a beautiful flower. It was so pretty and beautiful. Alice was so happy that she had found the flower. She hugged it and said, "Thank you, mommy! I love it!"
<|endoftext|>


In [9]:
import torch
ihvp = torch.load(f'/content/TinyStories_ihvp_33000000.pt')

In [10]:
ihvp.shape

torch.Size([10000, 248256])

In [14]:
ihvp[0].shape
248256 / 2048

121.21875

In [16]:
train_dataset = TinyStoriesDataset('train', tokenizer, block_size=2048, path='roneneldan/TinyStories')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [17]:
topk = 5

all_top_training_samples = []
all_top_influences = []
all_top_token_influences = []

test_dataloader = DataLoader(z_m, batch_size=1)

for query, compl in z_m:
    # torch.Tensor(list(filter(lambda x: x != 50256, query['input_ids'].tolist()))).long()
    print(ihvp.shape)
    print(query)
    grads = compute_grads(model, [(query, compl)])
    print("grads[0]", grads[0][0].view(-1).shape)
    print(len(grads))
    tokenwise_query_grads = grads
    for l in tokenwise_query_grads:
        print("token_query_grad", l[0].shape)
    query_grad = torch.cat(
        [q[0].view(-1) for q in grads]
    )
    print("query_grad", query_grad.shape)
    top_influences = -1 * torch.einsum("ij,j->i", ihvp, query_grad)
    print("top_influences", top_influences.shape)

    top_influences, top_samples = torch.topk(top_influences, topk)
    print(top_samples)
    all_top_training_samples.append(top_samples)
    all_top_influences.append(top_influences)
    all_top_token_influences.append(1)
    break

def decode(token_ids):
    try:
        return "".join([chr(i) for i in token_ids])
    except:
        return chr(token_ids)

# TODO ensure we are doing this per token when using real data
for i, (top_samples, top_influences, top_token_influences) in enumerate(
        zip(all_top_training_samples, all_top_influences, all_top_token_influences)
    ):
        print(f"Query: {tokenizer.decode(z_m[i][0]['input_ids'])}")
        print(f"Top {topk} training samples and their influences:")
        for s, i in zip(top_samples, top_influences):
            s = s.item()
            sample = f"{tokenizer.decode(train_dataset[s][0]['input_ids'])}"
            print(
                f"###\n{sample}\n Influence: {i}\n"
            )

torch.Size([10000, 248256])
{'input_ids': tensor([ 7454,  2402,   257,  ..., 50256, 50256, 50256], device='cuda:0')}
grads[0] torch.Size([16640])
15
token_query_grad torch.Size([256, 65])
token_query_grad torch.Size([64, 257])
token_query_grad torch.Size([256, 65])
token_query_grad torch.Size([64, 257])
token_query_grad torch.Size([256, 65])
token_query_grad torch.Size([64, 257])
token_query_grad torch.Size([256, 65])
token_query_grad torch.Size([64, 257])
token_query_grad torch.Size([256, 65])
token_query_grad torch.Size([64, 257])
token_query_grad torch.Size([256, 65])
token_query_grad torch.Size([64, 257])
token_query_grad torch.Size([256, 65])
token_query_grad torch.Size([64, 257])
token_query_grad torch.Size([256, 65])
query_grad torch.Size([248256])
top_influences torch.Size([10000])
tensor([9374, 9105, 9784, 5136, 1203], device='cuda:0')
Query: Once upon a time, there was a girl name Alice. Alice enjoyed many things. One of the things Alice enjoyed was playing outside. She was s